In [1]:
import sys
sys.path.append('./models')
from models.BBSNet_model import BBSNetTransformerAttention
from models.BBSNet_model import BBSNetChannelSpatialAttention
from BBSNet_train import setup_gpu, get_model, get_data, main
from BBSNet_test import test
from utils import evaluate, draw_curve

%load_ext autoreload
%autoreload 2

/home/mahdi/Documents/pytorch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# from models.RFNet import RFNet
# import torch

# rgb = torch.rand((2, 3, 352, 352)).cuda()
# depth = torch.rand((2, 1, 352, 352)).cuda()
# model = RFNet(32, 50).cuda()
# l = model(rgb, depth)

# Train

In [3]:
setup_gpu()
model, optimizer, CE = get_model()
train_loader, test_loader, total_step = get_data()
main(optimizer, model, train_loader, test_loader, CE, total_step)

USE GPU 0


/home/mahdi/Documents/pytorch/lib/python3.12/site-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


load data...
Start train...


/home/mahdi/Documents/pytorch/lib/python3.12/site-packages/torch/nn/_reduction.py:51: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


2025-09-01 00:02:13.464347 Epoch [001/002], Step [0001/2184], Loss1: 1.6055 Loss2: 1.5175
2025-09-01 00:02:29.094119 Epoch [001/002], Step [0100/2184], Loss1: 0.6381 Loss2: 0.9668
2025-09-01 00:02:44.924989 Epoch [001/002], Step [0200/2184], Loss1: 0.9019 Loss2: 0.8982
2025-09-01 00:03:00.838146 Epoch [001/002], Step [0300/2184], Loss1: 0.9443 Loss2: 1.2521
Keyboard Interrupt: save model and exit.
save checkpoints successfully!


KeyboardInterrupt: 

# Use Trained Model

I save the model using this code:
torch.save(model.state_dict(), save_path +
                           'BBSNet_epoch_best.pth')

Now, after a long training period, I want to use it on my test data. So, how can I load this model?

In [ ]:
from models.BBSNet_model import BBSNetTransformerAttention

model = BBSNetTransformerAttention()

# assume your model is called `model`
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")
print(f"Non-trainable parameters: {total_params - trainable_params}")


Total parameters: 60894418
Trainable parameters: 60894418
Non-trainable parameters: 0


In [ ]:
test(method='swin_best') 

USE GPU 0


/home/mahdi/Documents/bach_prj/repo/BBS-Net/BBSNet_test.py:106: UserWarning: `nn.functional.upsample` is deprecated. Use `nn.functional.interpolate` instead.
  res = F.upsample(res, size=gt.shape,
[ WARN:0@29.871] global loadsave.cpp:848 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.


save img to:  ./pred/swin_best/NJU2K/000008_left.png
save img to:  ./pred/swin_best/NJU2K/000013_left.png
save img to:  ./pred/swin_best/NJU2K/000017_left.png
save img to:  ./pred/swin_best/NJU2K/000023_left.png
save img to:  ./pred/swin_best/NJU2K/000025_left.png
save img to:  ./pred/swin_best/NJU2K/000026_left.png
save img to:  ./pred/swin_best/NJU2K/000027_left.png
save img to:  ./pred/swin_best/NJU2K/000028_left.png
save img to:  ./pred/swin_best/NJU2K/000034_left.png
save img to:  ./pred/swin_best/NJU2K/000040_left.png
save img to:  ./pred/swin_best/NJU2K/000043_left.png
save img to:  ./pred/swin_best/NJU2K/000084_left.png
save img to:  ./pred/swin_best/NJU2K/000087_left.png
save img to:  ./pred/swin_best/NJU2K/000089_left.png
save img to:  ./pred/swin_best/NJU2K/000090_left.png
save img to:  ./pred/swin_best/NJU2K/000097_left.png
save img to:  ./pred/swin_best/NJU2K/000100_left.png
save img to:  ./pred/swin_best/NJU2K/000110_left.png
save img to:  ./pred/swin_best/NJU2K/000111_le